In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

The task is to complete a grid-based function $y_i = f(x_i)$, $i = \overline{0, n}$, $x_i \in [a,b]$
on the basis of Cubic Spline with $q=1$, meaning

$S_3 \in C_2[a, b]$

and

$S_{3, i} = a_{0, i} + a_{1, i} (x - x_i) + a_{2, i} (x - x_i)^2 + a_{3, i} (x - x_i)^3 $

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Functional compatibility conditions:
1. $\delta S_{3, i}(x)$ $\large|_{x=x_i}$ $= S_{3, i}(x_i)$ $ - f(x_i)$ $ = 0$
2. $\delta S_{3, i}(x)$ $\large|_{x=x_{i+1}}$ $= S_{3, i}(x_{i+1})$ $ - f(x_{i+1})$ $ = 0$

Differential compatibility conditions:
3. $\delta S^{''}_{3, i}(x)$ $\large|_{x=x_i}$ $= S^{''}_{3, i}(x_i)$ $ - f^{''}(x_i)$ $ = 0$
4. $\delta S^{''}_{3, i}(x)$ $\large|_{x=x_{i+1}}$ $= S^{''}_{3, i}(x_{i+1})$ $ - f^{''}(x_{i+1})$ $ = 0$

### 3. Formulate stitching conditions [mathematical formula]

$S^{'}_{3, i-1}(x)$ $\large|_{x=x_i} = S^{'}_{3, i}(x)$ $\large|_{x=x_i}$  $ i = \overline{1, n-1}$


### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

The fuction is in smoothness  $ C_2[a,b]$, which means that the first and second derivatives are continuos. 

Functional and diffirential conditions provide us with continuity in the spline (up to the second derivative), and stitching conditions provie us continuity at the end of the splines.



### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

Spline eqaution: $S_i(x) = a_{0,i} + a_{1,i}(x - x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$

Compatibility conditions: 
1. $ S_{3, i}(x_i)$ $ - f(x_i)$ $ = 0$
2. $S_{3, i}(x_{i+1})$ $ - f(x_{i+1})$ $ = 0$
3. $S^{''}_{3, i}(x_i)$ $ - f^{''}(x_i)$ $ = 0$
4. $ S^{''}_{3, i}(x_{i+1})$ $ - f^{''}(x_{i+1})$ $ = 0$

First derivative of $S_i(x)$:

$S'_{3, i}(x) = a_{1,i} + 2a_{2, i}(x - x_i) + 3a_{3, i}(x - x_i)^2$

Second derivative of $S_i(x)$:

$S''_{3, i}(x) = 2a_{2, i} + 6a_{3, i} (x - x_i)$

Substituting each equation of Compatibility conditions with Spline eqaution and the second derivative of $S_i(x)$:
1. $ a_{0i} + a_{1,i}(x_i - x_i) + a_{2i}(x_i-x_i)^2 + a_{3i}(x_i-x_i)^3- f(x_i) = 0$
2. $a_{0,i} + a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2 + a_{3,i}(x_{i+1} - x_i)^3 - f(x_{i+1}) = 0 $
3. $2a_{2, i} + 6a_{3, i} (x_i - x_i) - f^{''}(x_i) = 0$
4. $ 2a_{2, i} + 6a_{3, i} (x_{i+1} - x_i) - f^{''}(x_{i+1}) = 0$

Then:
1. $ a_{0i} - f(x_i) = 0$
2. $a_{0,i} + a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2 + a_{3,i}(x_{i+1} - x_i)^3 - f(x_{i+1}) = 0 $
3. $2a_{2, i}  - f^{''}(x_i) = 0$
4. $ 2a_{2, i} + 6a_{3, i} (x_{i+1} - x_i) - f^{''}(x_{i+1}) = 0$


Substracting (3) from (4):

5.$6a_{3, i} (x_{i+1} - x_i) +  f^{''}(x_i) -  f^{''}(x_{i+1}) = 0 $

Given that $h_{i+1} = x_{i+1} - x_{i}$, $\Delta f_i = f_{i+1} - f_{i}$, $f_i = f(x_i)$, $\Delta m_i = m_{i+1} - m_i$, $m_i = f^{''}(x_i)$

$6a_{3, i}h_{i+1} - \Delta m_i = 0$

Therefore: $a_{3, i} = \frac{1}{6h_{i+1}}\Delta m_i$

Therefore:
1. $ a_{0i} = f(x_i)$
2. $a_{0,i} + a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2 + a_{3,i}(x_{i+1} - x_i)^3 - f(x_{i+1}) = 0 $
3. $a_{2, i} = \frac{m_i}{2}$
4. $a_{3, i} = \frac{1}{6h_{i+1}}\Delta m_i$

Substituting all the known variables in (2) and solving it for $a_{1, i}$ :

$f_i + a_{1,i}h_{i+1} + \frac{m_i}{2} h_{i+1}^2 + \frac{1}{6h_{i+1}}\Delta m_i h_{i+1}^3 - f_{i+1} = 0$
$a_{1, i} = \frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2} m_i - \frac{h_{i+1}}{6}\Delta m_i$

Therefore the coefficients become:
1. $ a_{0i} = f(x_i)$
2. $a_{1, i} = \frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2} m_i - \frac{h_{i+1}}{6}\Delta m_i$
3. $a_{2, i} = \frac{m_i}{2}$
4. $a_{3, i} = \frac{1}{6h_{i+1}}\Delta m_i$

The formaula then becomes:
$  S_{3,i}(x) = f(x_i) 
        + ( \frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2} m_i - \frac{h_{i+1}}{6}\Delta m_i) (x-x_i)
        + \frac{m_i}{2} (x-x_i)^2 +  \frac{1}{6h_{i+1}}\Delta m_i (x-x_i)^3.$
        
Using stiching conditions and the first derivative:

$  S_{3, i-1}^{'}(x) = a_{1, i-1} + 2a_{2, i-1}(x - x_{i-1}) + 3a_{3, i-1}(x - x_{i-1})^2 $ 


$ S_{3, i}^{'}(x) = a_{1, i} + 2a_{2, i}(x - x_{i}) + 3a_{3, i}(x - x_{i})^2$ 

Substitute coeffitients into stitching condition:
$  \frac{1}{h_i} \Delta f_{i-1} - \frac{h_i}{2} m_{i-1} - \frac{h_i}{6} \Delta m_{i-1} +  m_{i-1}(x_i - x_{i-1}) + \frac{\Delta m_{i-1}}{2h_{i}}(x_i - x_{i-1})^2 = \frac{1}{h_{i+1}} \Delta f_{i} - \frac{h_{i+1}}{2} m_{i} - \frac{h_{i+1}}{6} \Delta m_{i} +  m_{i}(x_i - x_{i}) + \frac{\Delta m_{i}}{2h_{i+1}}(x_i - x_{i})^2 $ 

Simplifying:

$\frac{h_i}{6}m_{i-1} + \frac{h_i + h_{i+1}}{3} m_i + \frac{h_{i+1}}{6} m_{i+1} =  \frac{1}{h_{i+1}} \Delta f_i - \frac{1}{h_i} \Delta f_{i-1}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$$\begin{bmatrix} \frac{h_1}{6}&
\frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & 0 & \dots & 0 & 0 & 0 & \frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\ 0 &
\frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & 0 & \dots & 0 & 0 & 0 & \frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\ 0 &
0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \frac{h_4}{6} & \dots & 0 & 0 & 0 & \frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\ \dots &
\dots & \dots & \dots & \dots & \dots & \dots & \dots & \dots & \dots \\ 0 &
0 & 0 & 0 & 0 & \dots & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_{n}}{3} & \frac{h_n}{6} & \frac{\Delta f_{n-1}}{h_{n}} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{bmatrix} $$


### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

In this system an unknown vaiable is  $m_i$. The system is not closed with respect to this unknown variable. For closure we should understand its value on the ends of the spline. We could assume that $m_0 = 0$, 
$m_n = 0$ to obtain a natural spline.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$$
\begin{bmatrix}
1 & \frac{h_2}{2(h_1 + h_2)} & 0 & 0 & \dots & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1}) \frac{3}{h_1 + h_2}  \\
0 & 1 & \frac{h_3}{2(h_2 + h_3)} - \frac{2 h_3 (h_1 + h_2)}{h_2^2} & 0 & \dots & (\frac{h_2}{2(h_1 + h_2)} (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1}) - \frac{\Delta f_2}{h_3} + \frac{\Delta f_1}{h_2}) : (-\frac{h_2 + h_3}{3} + \frac{h_2^2}{12(h_1 + h_2)}) \\
\dots & \dots & \dots & \dots & \dots & \dots \\
0 & 0 & 0 & 1 & \dots & (\frac{h_{n-1}}{6} Q_{n-2}) : (-\frac{h_{n-1} + h_n}{3} - \frac{h_{n-1}}{6} P_{n-2})
\end{bmatrix}
$$ 

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Direct pass:

1. Find $P_1$ and $Q_1$

    From 8th exercise:
    $$-P_1 = -\frac{\gamma_1}{\beta_1} = \frac{h_2}{2(h_1 + h_2)}$$
    $$P_1 = -\frac{h_2}{2(h_1 + h_2)}$$
    $$Q_1 = -\frac{\delta_1}{\beta_1} = -(\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1}) \frac{3}{h_1 + h_2}$$

2. Find coefficients $P_2, Q_2$; $P_3, Q_3$; $\dots$; $P_{n-1}, Q_{n-1}$, where

$$-P_2 = \gamma_2 : (-\beta_2 + \frac{\gamma_1 \alpha_2}{\beta_1}) =  \gamma_2 : (-\beta_2 + \alpha_2 \frac{\gamma_1}{\beta_1})$$

Substitute $P_1$:

$$P_2 = \frac{\gamma_2}{\beta_2 - \alpha_2 P_{1}}$$

Thus,

$P_i = \frac{\gamma_i}{\beta_i - \alpha_i P_{i-1}} = \frac{h_{i+1}}{6} : (-\frac{h_i + h_{i+1}}{3} - \frac{h_i}{6} P_{i-1}) = - \frac{h_{i+1}}{2(h_i + h_{i+1})} - \frac{h_{i+1}}{h_i} P_{i-1}$

$$Q_2 = (\delta_2 + \frac{\delta_1 \alpha_2}{\beta_1}) : (-\beta_2 + \frac{\gamma_1 \alpha_2}{\beta_1}) = (\delta_2 - \alpha_2 (-\frac{\delta_1}{\beta_1})) : (-\beta_2 + \alpha_2 \frac{\gamma_1}{\beta_1})$$

Substitute $P_1$ and $Q_1$:

$$Q_2 = \frac{\delta_2 - \alpha_2 Q_1}{-\beta_2 + \alpha_2 P_1} = \frac{\alpha_2 Q_{1} - \delta_2}{\beta_2 - \alpha_2 P_{1}}$$

Thus,

$Q_i = \frac{\alpha_i Q_{i-1} - \delta_i}{\beta_i - \alpha_i P_{i-1}} = (\frac{h_i}{6} Q_{i-1} - \frac{\Delta f_i}{h_{i+1}} + \frac{\Delta f_{i-1}}{h_i}) : (- \frac{h_i + h_{i+1}}{3} - \frac{h_i}{6} P_{i-1})$

Reverse pass:

1. Find $m_{n-1} = \frac{\alpha_{n-1} Q_{n-2} - \delta_{n-1}}{\beta_{n-1} - \alpha_{n-1} P_{n-2}} = (\frac{h_{n-1}}{6} Q_{n-2} - \frac{\Delta f_{n-1}}{h_{n}} + \frac{\Delta f_{n-2}}{h_{n-1}}) : (- \frac{h_{n-1} + h_{n}}{3} - \frac{h_{n-1}}{6} P_{n-2})$

2. Find $m_{n-2}, m_{n-3}, \dots, m_1$ with formula 

$m_{n-2} = P_{n-2} m_{n-1} + Q_{n-2} = P_{n-2} (\frac{h_{n-1}}{6} Q_{n-2} - \frac{\Delta f_{n-1}}{h_{n}} + \frac{\Delta f_{n-2}}{h_{n-1}}) : (- \frac{h_{n-1} + h_{n}}{3} - \frac{h_{n-1}}{6} P_{n-2}) + Q_{n-2}$

$m_{n-3} = P_{n-3} m_{n-2} + Q_{n-3}$

$m_1 = P_1 m_2 + Q_1$


### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [0]:
def create_matrix():
    pass

def direct_pass():
    pass
  
def reverse_pass():
    pass

def combine():
    pass










### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Assuming $u(x) \in C_4[a,b]$, 

Cubic Spline error:

$|| f^{(p)}(x) - S_3^{(p)}(x) ||_{C[a,b]} = \max \limits_{[a,b]} |  f^{(p)}(x) - S_3^{(p)}(x) | \leq M_4 h^{4-p}, p = 0, 1, 2$

For function:

$|| f(x) - S_3(x) ||_{C[a,b]} = \max \limits_{[a,b]} |  f(x) - S_3(x) | \leq M_4 h^{4}$

For first derivative:

$|| f^{'}(x) - S_3^{'}(x) ||_{C[a,b]} = \max \limits_{[a,b]} |  f^{'}(x) - S_3^{'}(x) | \leq M_4 h^{3}$

For second derivative:

$|| f^{''}(x) - S_3^{''}(x) ||_{C[a,b]} = \max \limits_{[a,b]} |  f^{''}(x) - S_3^{''}(x) | \leq M_4 h^{2}$


### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

For Direct pass: $O(n)$

For Reverse pass: $O(n)$

Total: $O(n)$

### Congrats!